# Control1 - Empirical fMRI control analysis, controlling for motion in task versus rest comparisons 

## Takuya Ito
#### 04/12/2019

In [174]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
import seaborn as sns
plt.style.use("ggplot")
sns.set_style('whitegrid')
plt.rcParams["font.family"] = "FreeSans"
import nibabel as nib
import pandas as pd
import sys
sys.path.append('glmScripts/')
import taskGLMPipeline as tgp
import h5py
import dimensionality
sys.path.append('bctpy/')
import bct

## Set-up subject list and construct global variables

In [175]:
## Exploratory subjects
subjNums = ['178950','189450','199453','209228','220721','298455','356948','419239','499566','561444','618952','680452','757764','841349','908860',
            '103818','113922','121618','130619','137229','151829','158035','171633','179346','190031','200008','210112','221319','299154','361234',
            '424939','500222','570243','622236','687163','769064','845458','911849','104416','114217','122317','130720','137532','151930','159744',
            '172029','180230','191235','200614','211316','228434','300618','361941','432332','513130','571144','623844','692964','773257','857263',
            '926862','105014','114419','122822','130821','137633','152427','160123','172938','180432','192035','200917','211417','239944','303119',
            '365343','436239','513736','579665','638049','702133','774663','865363','930449','106521','114823','123521','130922','137936','152831',
            '160729','173334','180533','192136','201111','211619','249947','305830','366042','436845','516742','580650','645450','715041','782561',
            '871762','942658','106824','117021','123925','131823','138332','153025','162026','173536','180735','192439','201414','211821','251833',
            '310621','371843','445543','519950','580751','647858','720337','800941','871964','955465','107018','117122','125222','132017','138837',
            '153227','162329','173637','180937','193239','201818','211922','257542','314225','378857','454140','523032','585862','654350','725751',
            '803240','872562','959574','107422','117324','125424','133827','142828','153631','164030','173940','182739','194140','202719','212015',
            '257845','316633','381543','459453','525541','586460','654754','727553','812746','873968','966975']

# # Validation subjects
# subjNums = ['100206','108020','117930','126325','133928','143224','153934','164636','174437','183034','194443','204521','212823','268749','322224',
#             '385450','463040','529953','587664','656253','731140','814548','877269','978578','100408','108222','118124','126426','134021','144832',
#             '154229','164939','175338','185139','194645','204622','213017','268850','329844','389357','467351','530635','588565','657659','737960',
#             '816653','878877','987074','101006','110007','118225','127933','134324','146331','154532','165638','175742','185341','195445','205119',
#             '213421','274542','341834','393247','479762','545345','597869','664757','742549','820745','887373','989987','102311','111009','118831',
#             '128632','135528','146432','154936','167036','176441','186141','196144','205725','213522','285345','342129','394956','480141','552241',
#             '598568','671855','744553','826454','896879','990366','102513','112516','118932','129028','135629','146533','156031','167440','176845',
#             '187850','196346','205826','214423','285446','348545','395756','481042','553344','599671','675661','749058','832651','899885','991267',
#             '102614','112920','119126','129129','135932','147636','157336','168745','177645','188145','198350','208226','214726','286347','349244',
#             '406432','486759','555651','604537','679568','749361','835657','901442','992774','103111','113316','120212','130013','136227','148133',
#             '157437','169545','178748','188549','198451','208327','217429','290136','352738','414229','497865','559457','615744','679770','753150',
#             '837560','907656','993675','103414','113619','120414','130114','136833','150726','157942','171330']

basedir = '/projects3/TaskFCMech/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

tasks = {'EMOTION':[0,1],
     'GAMBLING':[2,3],
     'LANGUAGE':[4,5],
     'MOTOR':[6,7,8,9,10,11],
     'RELATIONAL':[12,13],
     'SOCIAL':[14,15],
     'WM':[16,17,18,19,20,21,22,23]}
taskNames = ['EMOTION', 'GAMBLING', 'LANGUAGE', 'MOTOR', 'RELATIONAL', 'SOCIAL', 'WM']
TRsPerRun = [176,176,253,253,316,316,284,284,232,232,274,274,405,405]
restRuns = ['rfMRI_REST1_RL', 'rfMRI_REST1_LR','rfMRI_REST2_RL', 'rfMRI_REST2_LR']
taskRuns= ['tfMRI_EMOTION_RL','tfMRI_EMOTION_LR','tfMRI_GAMBLING_RL','tfMRI_GAMBLING_LR',
           'tfMRI_LANGUAGE_RL','tfMRI_LANGUAGE_LR','tfMRI_MOTOR_RL','tfMRI_MOTOR_LR',
           'tfMRI_RELATIONAL_RL','tfMRI_RELATIONAL_LR','tfMRI_SOCIAL_RL','tfMRI_SOCIAL_LR','tfMRI_WM_RL','tfMRI_WM_LR']

# 0.0 Define functions for loading motion spike data

In [176]:
def loadMotionData(subj, task_timing):
    
    task_motion_spikes = []
    rest_motion_spikes = []
    datafile = '/projects3/TaskFCMech/data/hcppreprocessedmsmall/nuisanceRegressors/' + subj + '_nuisanceRegressors.h5' 
    h5f = h5py.File(datafile,'r')
    
    runcount = 0
    for run in taskRuns:
        try:
            # skip first 5 TRs
            task_motion_spikes.extend(np.sum(h5f[run]['motionSpikes'][5:].copy(),axis=1))
        except:
            task_motion_spikes.extend(np.zeros((TRsPerRun[runcount]-5,),dtype=bool))
        runcount += 1
            
    for run in restRuns:
        try:
            # Skip first 5 TRs
            rest_motion_spikes.extend(np.sum(h5f[run]['motionSpikes'][5:].copy(),axis=1))
        except:
            rest_motion_spikes.extend(np.zeros((1195,),dtype=bool))
    
    h5f.close()
    
    rest_motion_spikes = np.asarray(rest_motion_spikes)
    task_motion_spikes = np.asarray(task_motion_spikes)

    avg_motion_rest = np.mean(rest_motion_spikes[:len(task_timing)][task_timing])
    avg_motion_task = np.mean(task_motion_spikes[task_timing])
    
    return avg_motion_rest, avg_motion_task
            
    

## Load task timing and calculate average rest and task motion spikes for Exploratory cohort

In [177]:
nTasks = 24
task_timing = np.zeros((3810,nTasks,len(subjNums)))
scount = 0
for subj in subjNums:
    if scount%25==0: print 'Loading in data for subject', scount+1, '/', len(subjNums)
    task_timing[:,:,scount] = tgp.loadTaskTimingForAllTasks(subj,taskModel='canonical')['taskRegressors']>.5 # Only to obtain which time points to estimate FC with
    scount += 1
    
task_timing = task_timing.astype(bool)


Loading in data for subject 1 / 176
Loading in data for subject 26 / 176
Loading in data for subject 51 / 176
Loading in data for subject 76 / 176
Loading in data for subject 101 / 176
Loading in data for subject 126 / 176
Loading in data for subject 151 / 176
Loading in data for subject 176 / 176


#### Calculate motion spikes for rest and task

In [178]:
task_timing_all = np.sum(task_timing,axis=1) > 0
restmotion = []
taskmotion = []
scount = 0
for subj in subjNums:
    rest,task = loadMotionData(subj, task_timing_all[:,scount])
    restmotion.append(rest)
    taskmotion.append(task)
    scount += 1

In [179]:
print 'Average percentage of motion spikes for rest:', np.mean(restmotion)*100.0
print 'Average percentage of motion spikes for task:', np.mean(taskmotion)*100.0
print 'Task v rest percentage diff for motion spikes:', np.mean(taskmotion)*100.0 - np.mean(restmotion)*100.0
t, p = stats.ttest_rel(taskmotion,restmotion)
print 'Task V Rest'
print '\tt =', t
print '\tp =', p

Average percentage of motion spikes for rest: 0.8113834999874038
Average percentage of motion spikes for task: 0.9092657615022246
Task v rest percentage diff for motion spikes: 0.09788226151482082
Task V Rest
	t = 1.0784408739670803
	p = 0.2823212896732061


## Load task timing and calculate average rest and task motion spikes for Replication cohort

In [180]:
# Validation subjects
subjNums = ['100206','108020','117930','126325','133928','143224','153934','164636','174437','183034','194443','204521','212823','268749','322224',
            '385450','463040','529953','587664','656253','731140','814548','877269','978578','100408','108222','118124','126426','134021','144832',
            '154229','164939','175338','185139','194645','204622','213017','268850','329844','389357','467351','530635','588565','657659','737960',
            '816653','878877','987074','101006','110007','118225','127933','134324','146331','154532','165638','175742','185341','195445','205119',
            '213421','274542','341834','393247','479762','545345','597869','664757','742549','820745','887373','989987','102311','111009','118831',
            '128632','135528','146432','154936','167036','176441','186141','196144','205725','213522','285345','342129','394956','480141','552241',
            '598568','671855','744553','826454','896879','990366','102513','112516','118932','129028','135629','146533','156031','167440','176845',
            '187850','196346','205826','214423','285446','348545','395756','481042','553344','599671','675661','749058','832651','899885','991267',
            '102614','112920','119126','129129','135932','147636','157336','168745','177645','188145','198350','208226','214726','286347','349244',
            '406432','486759','555651','604537','679568','749361','835657','901442','992774','103111','113316','120212','130013','136227','148133',
            '157437','169545','178748','188549','198451','208327','217429','290136','352738','414229','497865','559457','615744','679770','753150',
            '837560','907656','993675','103414','113619','120414','130114','136833','150726','157942','171330']

In [181]:
nTasks = 24
task_timing = np.zeros((3810,nTasks,len(subjNums)))
scount = 0
for subj in subjNums:
    if scount%25==0: print 'Loading in data for subject', scount+1, '/', len(subjNums)
    task_timing[:,:,scount] = tgp.loadTaskTimingForAllTasks(subj,taskModel='canonical')['taskRegressors']>.5 # Only to obtain which time points to estimate FC with
    scount += 1
    
task_timing = task_timing.astype(bool)


Loading in data for subject 1 / 176
Loading in data for subject 26 / 176
Loading in data for subject 51 / 176
Loading in data for subject 76 / 176
Loading in data for subject 101 / 176
Loading in data for subject 126 / 176
Loading in data for subject 151 / 176
Loading in data for subject 176 / 176


#### Calculate motion spikes for rest and task

In [182]:
task_timing_all = np.sum(task_timing,axis=1) > 0
restmotion = []
taskmotion = []
scount = 0
for subj in subjNums:
    rest,task = loadMotionData(subj, task_timing_all[:,scount])
    restmotion.append(rest)
    taskmotion.append(task)
    scount += 1

In [183]:
print 'Average number of motion spikes for rest:', np.mean(restmotion)
print 'Average number of motion spikes for task:', np.mean(taskmotion)
t, p = stats.ttest_rel(taskmotion,restmotion)
print 'Task V Rest'
print '\tt =', t
print '\tp =', p

Average number of motion spikes for rest: 0.007563852098834378
Average number of motion spikes for task: 0.00886632003422832
Task V Rest
	t = 1.5264817968200626
	p = 0.12869523271120606
